<a href="https://colab.research.google.com/github/VaibhavNagrale007/Adversarial-Machine-Learning-BTP/blob/main/Sem%207/red_attack_xray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I am sharing the implementation of the RED attack on the CNN model using the GTSRB dataset. You can download that dataset from the internet and replicate the results. Also, implement the same in medical images for CNN and a Vision Transformer of your choice.

In [ ]:
!pip install Augmentor

In [ ]:
import Augmentor
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import os, shutil, glob, os.path
import cv2
import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
train_imdir = '/content/drive/MyDrive/Dataset/xray_pneumonia/train/'
test_imdir = '/content/drive/MyDrive/Dataset/xray_pneumonia/test/'
val_imdir = '/content/drive/MyDrive/Dataset/xray_pneumonia/val/'

r = Augmentor.Pipeline(val_imdir)
r.flip_left_right(0.5)
r.rotate(0.3, 10, 10)
r.skew(0.4, 0.5)
r.zoom(probability = 0.2, min_factor = 1.1, max_factor = 1.5)
r.sample(500)

p = Augmentor.Pipeline(train_imdir)
p.flip_left_right(0.5)
p.rotate(0.3, 10, 10)
p.skew(0.4, 0.5)
p.zoom(probability = 0.2, min_factor = 1.1, max_factor = 1.5)
p.sample(500)

Initialised with 16 image(s) found.
Output directory set to /content/drive/MyDrive/Dataset/xray_pneumonia/val/output.

Processing <PIL.Image.Image image mode=L size=1776x1416 at 0x7AFC4C245570>: 100%|██████████| 500/500 [00:42<00:00, 11.73 Samples/s]


KeyboardInterrupt: ignored

In [ ]:
def img_labels(path):
  images = []
  img_cnt = 0
  imdir = path + 'output/NORMAL'
  filelist = glob.glob(os.path.join(imdir, '*.jpeg'))
  if filelist == [] :
    imdir = path + 'NORMAL'
    filelist = glob.glob(os.path.join(imdir, '*.jpeg'))
  filelist.sort()
  for j, imagepath in enumerate(filelist):
      img1 = cv2.imread(filelist[j])
      img1 = cv2.resize(img1, (44, 44), interpolation=cv2.INTER_LINEAR)
      images.append(img1)
      img_cnt = img_cnt + 1
  imdir = path + 'output/PNEUMONIA'
  filelist = glob.glob(os.path.join(imdir, '*.jpeg'))
  if filelist == [] :
    imdir = path + 'PNEUMONIA'
    filelist = glob.glob(os.path.join(imdir, '*.jpeg'))
  filelist.sort()
  for j, imagepath in enumerate(filelist):
      img1= cv2.imread(filelist[j])
      img1 = cv2.resize(img1, (44, 44), interpolation=cv2.INTER_LINEAR)
      images.append(img1)

  labels = 0
  for i in range(1,len(images)):
    if(i<img_cnt):
      labels = np.vstack ((labels, 0) )
    else:
      labels = np.vstack ((labels, 1) )

  print("Number of images: ", len(images))
  print("Number of labels: ", len(labels))

  images=np.stack(images,axis=0)
  print("Images shape: ", images.shape)

  return images, labels

# train
train_images, train_labels = img_labels(train_imdir)

# val
val_images, val_labels = img_labels(val_imdir)

# test
test_images, test_labels = img_labels(test_imdir)

# Normalize pixel values to be between 0 and 1
train_images, test_images, val_images = train_images / 255.0, test_images / 255.0, val_images / 255.0

In [ ]:
# Increase image dimension and correct the code
# tp = test_images
# print("previous: ",tp)
# tp = tf.expand_dims(tp, axis=-1)
# print("new", tp)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(44, 44, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='sigmoid'))
model.add(layers.Dense(2, activation='softmax'))
model.summary()

import time
start_time = time.time()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10,
                    validation_data=(val_images, val_labels))
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#import dill
#dill.load_session('gtsrb.db')

In [ ]:
!pip install Augmentor
!unzip trainingbenign.zip
!unzip trainingmalign.zip
!unzip testbenign.zip
!unzip testmalign.zip

In [ ]:
#importing useful libraries
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import os, shutil, glob, os.path
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

import Augmentor
# Passing the path of the image directory
p = Augmentor.Pipeline("trainingbenign")
p.flip_left_right(0.5)
#p.black_and_white(0.1)
p.rotate(0.3, 10, 10)
p.skew(0.4, 0.5)
p.zoom(probability = 0.2, min_factor = 1.1, max_factor = 1.5)
p.sample(500)

q = Augmentor.Pipeline("trainingmalign")
q.flip_left_right(0.5)
#p.black_and_white(0.1)
q.rotate(0.3, 10, 10)
q.skew(0.4, 0.5)
q.zoom(probability = 0.2, min_factor = 1.1, max_factor = 1.5)
q.sample(750)

imdir = "trainingbenign"
filelist = glob.glob(os.path.join(imdir, '*.bmp'))
filelist.sort()
images=[]
for j, imagepath in enumerate(filelist):
    img1 = cv2.imread(filelist[j])
    img1 = cv2.resize(img1, (44, 44), interpolation=cv2.INTER_LINEAR)
    images.append(img1)
imdir = "trainingbenign/output"
filelist = glob.glob(os.path.join(imdir, '*.bmp'))
filelist.sort()
for j, imagepath in enumerate(filelist):
    img1= cv2.imread(filelist[j])
    img1 = cv2.resize(img1, (44, 44), interpolation=cv2.INTER_LINEAR)
    images.append(img1)
print(len(images))

imdir = "trainingmalign"
filelist = glob.glob(os.path.join(imdir, '*.bmp'))
filelist.sort()
for j, imagepath in enumerate(filelist):
    img1 = cv2.imread(filelist[j])
    img1 = cv2.resize(img1, (44, 44), interpolation=cv2.INTER_LINEAR)
    images.append(img1)
imdir = "trainingmalign/output"
filelist = glob.glob(os.path.join(imdir, '*.bmp'))
filelist.sort()
for j, imagepath in enumerate(filelist):
    img1= cv2.imread(filelist[j])
    img1 = cv2.resize(img1, (44, 44), interpolation=cv2.INTER_LINEAR)
    images.append(img1)
print(len(images))
labels = 0
for i in range(1,len(images)):
  if(i<580):
    labels = np.vstack ((labels, 0) )
  else:
    labels = np.vstack ((labels, 1) )

images=np.stack(images,axis=0)
print(images.shape)
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize pixel values to be between 0 and 1
train_images, val_images = train_images / 255.0, val_images / 255.0

imdir = "testbenign"
filelist = glob.glob(os.path.join(imdir, '*.bmp'))
filelist.sort()
images=[]
for j, imagepath in enumerate(filelist):
    img1 = cv2.imread(filelist[j])
    img1 = cv2.resize(img1, (44, 44), interpolation=cv2.INTER_LINEAR)
    images.append(img1)
print(len(images))
imdir = "testmalign"
filelist = glob.glob(os.path.join(imdir, '*.bmp'))
filelist.sort()
for j, imagepath in enumerate(filelist):
    img1 = cv2.imread(filelist[j])
    img1 = cv2.resize(img1, (44, 44), interpolation=cv2.INTER_LINEAR)
    images.append(img1)
print(len(images))
labels=0
for i in range(1,len(images)):
  if(i<20):
    labels = np.vstack ((labels, 0) )
  else:
    labels = np.vstack ((labels, 1) )
images=np.stack(images,axis=0)
print(images.shape)
test_images, test_labels = images,labels
test_images=test_images/255.0
print(len(test_images))
print(len(test_labels))

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(44, 44, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='sigmoid'))
model.add(layers.Dense(2, activation='softmax'))
model.summary()

import time
start_time = time.time()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10,
                    validation_data=(val_images, val_labels))
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
def max_diff(img1,img2):
    img = img1 - img2
    return np.amax(img)

def pred(image):
    data = []
    data.append(image)
    X_test = np.array(data)
    X_test = X_test.astype('float32')/255
    # pred = model.predict_classes(X_test)
    pred = np.argmax(model.predict(X_test),axis=1)
    return pred[0]

In [ ]:
def boundary_estimation(source, target, dmin):
    Ii = ((source + target)/2.0)
    k = pred(Ii)
    delta = max_diff(source, Ii)
    Ia2 = source
    Ib2 = target
    p = Ib2
    while (delta > dmin):
        if (pred(Ia2) != k):
            Ib2 = Ii
        else:
            Ia2 = Ii
        Ii = ((Ia2+Ib2)/2.0)
        k = pred(Ii)
        delta = max_diff(Ia2,Ii)
    return Ii

In [ ]:
def go_out(source,iout,alpha):
    i_diff = iout - source
    pred_source = pred(source)
    inew = iout
    while (pred(inew)==pred_source):
        inew = inew + alpha*(i_diff)

    return inew

In [ ]:
pip install -U setuptools

In [ ]:
!pip install preprocess

In [ ]:
from tensorflow.keras.preprocessing import image
def preprocess(img_path):
    # img = tf.expand_dims(img, axis=-1)
    # img = cv2.resize(img, (44, 44), interpolation=cv2.INTER_LINEAR)
    # return img
    img = image.load_img(img_path, target_size=(44, 44))
    img_array = image.img_to_array(img)
    # img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

In [ ]:
from PIL import Image
source_image_path = test_imdir + 'NORMAL/IM-0001-0001.jpeg';
target_image_path = test_imdir + 'PNEUMONIA/person100_bacteria_475.jpeg';
# img = (np.asarray(Image.open(source_image_path)))
# img1 = preprocess(img)
# source_image = np.array(img1)
# img = (np.asarray(Image.open(target_image_path)))
# img2 = preprocess(img)
# target_image = np.array(img2)
source_image = preprocess(source_image_path)
target_image = preprocess(target_image_path)

In [ ]:
print(source_image.shape)

In [ ]:
i = boundary_estimation(source_image,target_image,1.0)

In [ ]:
print (pred(i))
print (pred(source_image))
print (pred(target_image))

In [ ]:
ii = go_out(source_image,i,0.01)

In [ ]:
pred(ii)

NameError: ignored

In [ ]:
Image.fromarray(i.astype('uint8')).show()

In [ ]:
Image.fromarray(ii.astype('uint8')).show()

In [ ]:
def array_diff(d1):
    sumd1 = 0.0
    for i in range(0,3):
        for j in range(0,30):
            for k in range(0,30):
                d1[j][k][i] = d1[j][k][i]*d1[j][k][i]
                sumd1 = sumd1 + d1[j][k][i]
    return (sumd1)

def gradient_estimation(source, target, adversarial, n, theta):
    Ia = source
    Ib = target
    Ii = adversarial
    Io = np.zeros((2700))
    X = np.random.randint(0,2700, size=n)
    for i in X:
        Io[i] = 255
    Io = Io.reshape((30,30,3))
#     print(Io*theta)
    Ii2 = Ii + theta*Io
    Ii2_new = boundary_estimation(Ia, Ii2, 1.0)
    Ii2_new = go_out(source,Ii2_new,0.01)
    diff2 = Ii2_new - Ia
    diff1 = Ii - Ia
    d2 = array_diff(diff2)
    d1 = array_diff(diff1)
    if (d2 > d1):
        return (-1, Ii2_new)
    elif (d1 > d2):
        return (1, Ii2_new)
    else:
        return (0,Ii2_new)


In [ ]:
def efficient_update(source, target, adversarial, I2, g, j):
    Ia = source
    Ib = target
    Ii = adversarial
    Ii2 = I2
    delta = g*(Ii2 - Ii)
    l = j
    Inew = Ii + l*delta

    diff1 = Inew - Ia
    diff2 = Ii - Ia
    d1 = array_diff(diff1)
    d2 = array_diff(diff2)
    ii = 0
    it = 0
    while(d1 > d2):
        l = (l/2.0)
        Inew = Ii + l*delta
        if(pred(Inew)==pred(source)):
            Inew = go_out(source,Inew,0.01)
        it = it + 1
        d1 = array_diff(Inew-Ia)
        if(it>100):
            break
    if (d1 > d2):
        print(ii)
        ii = ii + 1
        Inew = Ii
    return Inew

In [ ]:
def iteration(itr, source, target, n, theta, j, dmin):
    targett = target
    sourcee = source
    for i in range(itr):
        #print (i)
        adversarial_image = boundary_estimation(sourcee, targett, dmin)
        adversarial_image = go_out(sourcee,adversarial_image,0.01)
        (g, Iii2) = gradient_estimation(sourcee, targett, targett, n, theta)
        targett = efficient_update(sourcee, targett, adversarial_image, Iii2, g, j)
        if (pred(targett) == pred(source)):
            j = j/2.0
        fin = targett
        if(pred(targett)==pred(sourcee)):
            fin = go_out(sourcee,targett,0.01)
        if(array_diff(fin-sourcee)<array_diff(adversarial_image-sourcee)):
            targett = fin
            #print("uopp")

    return fin

In [ ]:
final = iteration(300,source_image,target_image,5,0.196,5.0,1.0)

In [ ]:
source_image_path = 'gtsrb_dataset/kushbhinaamdede/0.png';
img = (np.asarray(Image.open(source_image_path)))
img1 = preprocess(img)
source = np.array(img1)

In [ ]:
arr = []
for i in range(1,43):
    print(i)
    target_image_pathh = 'gtsrb_dataset/kushbhinaamdede/{0}.png'.format(i)
    img = (np.asarray(Image.open(target_image_pathh)))
    img1 = preprocess(img)
    target = np.array(img1)
    f = iteration(100,source,target,5,0.196,5.0,1.0)
    arr.append(f)

1


/home/sakshi/dlProject/dlproject/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in ubyte_scalars
  


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


In [ ]:
for i in range(42):
    print(pred(arr[i]))

1
1
38
1
1
38
1
1
1
1
1
1
1
1
1
1
1
1
1
1
4
1
38
1
1
1
38
1
1
1
1
1
4
8
1
1
1
1
1
1
1
1


In [ ]:
Image.fromarray(source.astype('uint8')).save('oo.png')
Image.fromarray(arr[2].astype('uint8')).save('o0.png')
# s = measure.compare_ssim(arr[1],arr[0])

In [ ]:
import cv2

In [ ]:
original = cv2.imread("oo.png")
perturb = cv2.imread("o0.png")

In [ ]:
s = measure.compare_ssim(original,perturb,multichannel=True)
print(s)

0.8491636064596036


/home/sakshi/dlProject/dlproject/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  """Entry point for launching an IPython kernel.


In [ ]:
original = cv2.imread("oo.png")
for i in range(42):
    Image.fromarray(arr[i].astype('uint8')).save('o0.png')
    perturb = cv2.imread('o0.png')
    print(i,measure.compare_ssim(original,perturb,multichannel=True))

/home/sakshi/dlProject/dlproject/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  """


0 0.8638563696563702
1 0.8601731118867479
2 0.8491636064596036
3 0.5853071229217088
4 0.8793623636836144
5 0.725452119893038
6 0.7283911101181001
7 0.9110326737243134
8 0.846473262054997
9 0.898105590816233
10 0.7961759586655234
11 0.9037002428630313
12 0.4795590111181247
13 0.760121866147534
14 0.8513259876754575
15 0.911840161397691
16 0.7522741153556675
17 0.4316132464752868
18 0.7929031042224631
19 0.8122035694103497
20 0.8118060371115196
21 0.9376700961116479
22 0.48645602322363724
23 0.8156317982715976
24 0.4805004945690447
25 0.6643707643514745
26 0.6109604088463204
27 0.8220965324392212
28 0.808692374440188
29 0.7847599129101823
30 0.757587302481754
31 0.8200416075965494
32 0.8095893454733645
33 0.5888707728214264
34 0.5362472929834784
35 0.5623844175021794
36 0.8430409978867802
37 0.7293989226620735
38 0.4986993124303565
39 0.8271368793706709
40 0.8510807724897442
41 0.8131894513250736
